<img src="Images/Splice_logo.jpeg" width="250" height="200" align="left" >

# Welcome to Splice Machine!
## Splice Machine is powered by Spark, so the first step will be to initialize a Spark session. All Spark configuration is done for you here, but can be customized.

In [ ]:
#Begin spark session 
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#Create pysplice context. Allows you to create a Spark dataframe using our Native Spark DataSource 
from splicemachine.spark import PySpliceContext
splice = PySpliceContext(spark)

#Initialize our Feature Store API
from splicemachine.features import FeatureStore
from splicemachine.features.constants import FeatureType
fs = FeatureStore(splice)

#Initialize MLFlow
from splicemachine.mlflow_support import *
mlflow.register_feature_store(fs)
mlflow.register_splice_context(splice)

## The next step is to create a schema, twimlcon_fs, in the database to house your Feature Store
### This step has already been done for you

In [ ]:
%%sql
SELECT schemaname 
FROM SYS.SYSSCHEMAS;

## Next, you need to create a feature set
### A feature set coresponds to a data source. The user specifies the name of features inside of the feature set, and metadata associated with those features.

In [ ]:
# create customer lifetime metrics feature set
fs.create_feature_set(schema_name='twimlcon_fs',table_name='customer_lifetime', primary_keys = {"CUSTOMERID":"INTEGER"}, desc='Describes customer with aggregates of their lifetime activity.')

# add customer lifetime features

fs.create_feature(schema_name='twimlcon_fs',table_name='customer_lifetime', name='CUSTOMER_LIFETIME_ACTIVE_DAYS',            desc='Number of distinct days in which customer has placed orders.',           feature_data_type='INTEGER', feature_type=FeatureType.continuous,  tags='{"Entity":"Customer","Calculation":"Lifetime","MetricType":"Count","UnitOfMeasure":"Days","TimeGranularity":"Day"}')
fs.create_feature(schema_name='twimlcon_fs',table_name='customer_lifetime', name='CUSTOMER_LIFETIME_QTY',                    desc='Total lifetime number of units purchased.',                              feature_data_type='INTEGER', feature_type=FeatureType.continuous,  tags='{"Entity":"Customer","Calculation":"Lifetime","MetricType":"Sum","UnitOfMeasure":"Items","TimeGranularity":"Day"}')
fs.create_feature(schema_name='twimlcon_fs',table_name='customer_lifetime', name='CUSTOMER_LIFETIME_ITEMS_PER_ACTIVE_DAY',   desc='Total lifetime average number of units purchased per active day.',       feature_data_type='DOUBLE',  feature_type=FeatureType.continuous,  tags='{"Entity":"Customer","Calculation":"Lifetime","MetricType":"Average","UnitOfMeasure":"Items","TimeGranularity":"Day"}')
fs.create_feature(schema_name='twimlcon_fs',table_name='customer_lifetime', name='CUSTOMER_LIFETIME_REVENUE_PER_ACTIVE_DAY', desc='Total lifetime average revenue per active day.',                         feature_data_type='DOUBLE',  feature_type=FeatureType.continuous,  tags='{"Entity":"Customer","Calculation":"Lifetime","MetricType":"Average","UnitOfMeasure":"Dollars","TimeGranularity":"Day"}')
fs.create_feature(schema_name='twimlcon_fs',table_name='customer_lifetime', name='CUSTOMER_LIFETIME_DAYS',                   desc='Length of customer relationship in days.',                               feature_data_type='INTEGER', feature_type=FeatureType.continuous,  tags='{"Entity":"Customer","Calculation":"Lifetime","MetricType":"Duration","UnitOfMeasure":"Days","TimeGranularity":"Day"}')
fs.create_feature(schema_name='twimlcon_fs',table_name='customer_lifetime', name='CUSTOMER_DAYS_SINCE_PURCHASE',             desc='Days since customer made a purchase.',                                   feature_data_type='INTEGER', feature_type=FeatureType.continuous,  tags='{"Entity":"Customer","Calculation":"Lifetime","MetricType":"Duration","UnitOfMeasure":"Days","TimeGranularity":"Day"}')
fs.create_feature(schema_name='twimlcon_fs',table_name='customer_lifetime', name='CUSTOMER_LIFETIME_VALUE',                  desc='Total revenue over customer lifetime.',                                  feature_data_type='DOUBLE',  feature_type=FeatureType.continuous,  tags='{"Entity":"Customer","Calculation":"Lifetime","MetricType":"Sum","UnitOfMeasure":"Dollars","TimeGranularity":"Day"}')
fs.create_feature(schema_name='twimlcon_fs',table_name='customer_lifetime', name='CUSTOMER_START_DATE',                      desc='First invoice date for customer.',                                       feature_data_type='DATE',    feature_type=FeatureType.categorical, tags='{"Entity":"Customer","Calculation":"Lifetime","MetricType":"Min","UnitOfMeasure":"Date","TimeGranularity":"Day"}')

fs.deploy_feature_set('twimlcon_fs','customer_lifetime')

## When you create a Feature Set, you are creating two empty tables inside of the database that are connected by database triggers and functions

<img src="Images/FS_tables.png" width="750"  align="left" >

### These tables are:
- #### Feature table
- #### Feature History table

## Now you can add data to these tables. 
### First, add data to the feature table

In [ ]:
%%sql
TRUNCATE TABLE twimlcon_fs.customer_lifetime;
INSERT INTO twimlcon_fs.customer_lifetime
( CUSTOMERID, LAST_UPDATE_TS, CUSTOMER_LIFETIME_ACTIVE_DAYS,CUSTOMER_LIFETIME_QTY,
   CUSTOMER_LIFETIME_ITEMS_PER_ACTIVE_DAY,CUSTOMER_LIFETIME_REVENUE_PER_ACTIVE_DAY,
 CUSTOMER_LIFETIME_DAYS,CUSTOMER_DAYS_SINCE_PURCHASE, CUSTOMER_LIFETIME_VALUE,
 CUSTOMER_START_DATE)
SELECT CUSTOMERID, CAST(MAX(INVOICEDATE) AS TIMESTAMP) LAST_UPDATE_TS, -- use most recent event time for LAST_UPDATE_TS
    count(*) ACTIVE_DAYS, 
    sum(TOTAL_QTY) LIFETIME_QTY, 
    sum(TOTAL_QTY)*1.0/count(*) LIFETIME_ITEMS_PER_ACTIVE_DAY, 
    sum(TOTAL_REVENUE)*1.0/count(*) LIFETIME_REVENUE_PER_ACTIVE_DAY, 
    CURRENT_DATE - min(invoicedate) LIFETIME_DAYS, 
    CURRENT_DATE - max(invoicedate) DAYS_SINCE_PURCHASE,
    sum(TOTAL_REVENUE) LIFETIME_VALUE,
    min(invoicedate) CUSTOMER_START_DATE
FROM RETAIL_RFM.CUSTOMER_CATEGORY_ACTIVITY group by 1;

### Next, backfill the history table 

In [ ]:
%%sql
TRUNCATE TABLE twimlcon_fs.customer_lifetime_history;

-- backfill load of history values for customer lifetime features
INSERT INTO twimlcon_fs.customer_lifetime_history
( CUSTOMERID, ASOF_TS, UNTIL_TS, CUSTOMER_LIFETIME_ACTIVE_DAYS,CUSTOMER_LIFETIME_QTY,
  CUSTOMER_LIFETIME_ITEMS_PER_ACTIVE_DAY,CUSTOMER_LIFETIME_REVENUE_PER_ACTIVE_DAY,
  CUSTOMER_LIFETIME_DAYS,CUSTOMER_DAYS_SINCE_PURCHASE, 
  CUSTOMER_LIFETIME_VALUE, CUSTOMER_START_DATE)

SELECT * 
FROM
(
    SELECT CUSTOMERID, 
           CAST(THEDATE AS TIMESTAMP) ASOF_TS, 
           CAST(LAST_VALUE(THEDATE) OVER ( PARTITION BY CUSTOMERID ORDER BY THEDATE ROWS BETWEEN 1 FOLLOWING AND 1 FOLLOWING) AS TIMESTAMP) UNTIL_TS,
            ACTIVE_DAYS, LIFETIME_QTY, LIFETIME_ITEMS_PER_ACTIVE_DAY,LIFETIME_REVENUE_PER_ACTIVE_DAY,
            CUSTOMER_LIFETIME_DAYS, CUSTOMER_DAYS_SINCE_PURCHASE, LIFETIME_VALUE, LIFETIME_MIN_DATE
    FROM
    (
        SELECT x.CUSTOMERID, c.TheDate, ACTIVE_DAYS, LIFETIME_QTY, LIFETIME_ITEMS_PER_ACTIVE_DAY,LIFETIME_REVENUE_PER_ACTIVE_DAY, LIFETIME_VALUE,
               c.TheDate - LIFETIME_MIN_DATE as CUSTOMER_LIFETIME_DAYS, c.TheDate - LIFETIME_MAX_DATE as CUSTOMER_DAYS_SINCE_PURCHASE,
               LIFETIME_MIN_DATE
        FROM
            RETAIL_RFM.CALENDAR c 
        INNER JOIN
            (
                SELECT 
                    CUSTOMERID,
                    INVOICEDATE,
                    LAST_VALUE(INVOICEDATE) OVER (PARTITION BY CUSTOMERID ORDER BY INVOICEDATE ROWS BETWEEN 1 FOLLOWING AND 1 FOLLOWING) NEXT_INVOICEDATE,
                    count(*) OVER (PARTITION BY CUSTOMERID ORDER BY INVOICEDATE ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) ACTIVE_DAYS, 
                    sum(TOTAL_QTY) OVER (PARTITION BY CUSTOMERID ORDER BY INVOICEDATE ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)  LIFETIME_QTY, 
                    (sum(TOTAL_QTY) OVER (PARTITION BY CUSTOMERID ORDER BY INVOICEDATE ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)) *1.0/count(*)  OVER (PARTITION BY CUSTOMERID ORDER BY INVOICEDATE ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) LIFETIME_ITEMS_PER_ACTIVE_DAY, 
                    (sum(TOTAL_REVENUE)  OVER (PARTITION BY CUSTOMERID ORDER BY INVOICEDATE ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)) *1.0/count(*)  OVER (PARTITION BY CUSTOMERID ORDER BY INVOICEDATE ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) LIFETIME_REVENUE_PER_ACTIVE_DAY, 
                    min(invoicedate)  OVER (PARTITION BY CUSTOMERID ORDER BY INVOICEDATE ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) LIFETIME_MIN_DATE, 
                    max(invoicedate)  OVER (PARTITION BY CUSTOMERID ORDER BY INVOICEDATE ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) LIFETIME_MAX_DATE,
                    sum(TOTAL_REVENUE)  OVER (PARTITION BY CUSTOMERID ORDER BY INVOICEDATE ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) LIFETIME_VALUE
                FROM RETAIL_RFM.CUSTOMER_CATEGORY_ACTIVITY a --splice-properties splits=50
            )x
            ON c.TheDate BETWEEN x.INVOICEDATE AND x.NEXT_INVOICEDATE-1
    )x 
)y --splice-properties useSpark=true
WHERE UNTIL_TS IS NOT NULL
;
                     
--gather DB statistics
ANALYZE SCHEMA TWIMLCON_FS;

## Take a look at the Feature Store you just created

In [ ]:
%%sql
SELECT * 
FROM TWIMLCON_FS.CUSTOMER_LIFETIME
WHERE CUSTOMERID=12346;

SELECT * 
FROM TWIMLCON_FS.CUSTOMER_LIFETIME_HISTORY 
WHERE CUSTOMERID=12346 
ORDER BY ASOF_TS DESC 
{LIMIT 2};


# Now we have a fully functioning Feature Store!
- #### New data can be added directly to the feature table. As new data is added, old features will automatically move to the historical feature table.

## Update a value inside of the Feature Store

In [ ]:
%%sql
UPDATE TWIMLCON_FS.CUSTOMER_LIFETIME 
SET CUSTOMER_LIFETIME_ACTIVE_DAYS=11, 
LAST_UPDATE_TS = CURRENT_TIMESTAMP
WHERE CUSTOMERID=12346;

## Look at the new feature table and history table after the change

In [ ]:
%%sql
SELECT * 
FROM TWIMLCON_FS.CUSTOMER_LIFETIME
WHERE CUSTOMERID=12346;

SELECT * 
FROM TWIMLCON_FS.CUSTOMER_LIFETIME_HISTORY 
WHERE CUSTOMERID=12346 
ORDER BY ASOF_TS DESC 
{LIMIT 3};

In [ ]:
spark.stop()